# Introduction
### MNIST dataset
Handwritten digits dataset <br> 
Our goal here is to correctly identify digits from handwritten digits

# Importing the necessarry Libraries for Data Analysis
We'll be using the following libraries to perform data analysis 

In [ ]:
# Importing required libraries

import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Loading data and other preparations
We've loaded the data into training and testing datasets <br>
Further we've loaded the input and output (class) dataset as well

In [ ]:
mnist_train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
mnist_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
# Print mnist training dataset

mnist_train

In [ ]:
# Print mnist test dataset

mnist_test

# Understanding the Dataset
We clearly see that there are **42,000 training examples** and **28,000 testing examples** with a total of **60,000 grayscale data** as a part of the dataset <br>
Also, the images are of **28x28 pixels** flattened out into **an array of size 784**

# Design and Target
We'll now split the training and testing datasets into **design and target** <br>

In [ ]:
# For x_train, we just need to drop the "labels" column
# Axis is by default 0 in .drop meaning that by default row gets dropped
# Axis = 1 here means that we're dropping a column

x_train = mnist_train.drop(labels = "label", axis = 1)
x_train

In [ ]:
# For y_train we just need the "labels" column

y_train = mnist_train["label"]
y_train

# Normalization
Normalization is required to bring out dataset to a **common scale**

In [ ]:
x_train = x_train/255.0
mnist_test = mnist_test/255.0

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
print(mnist_test.shape)

# Resizing the 1D matrix into a 3D matrix
Here, we are transforming the **1D matrix** of size **784** to a **3D matrix** of size **28x28x1**

In [ ]:
# -1 means that we want to keep all the values, that is the initial size intact
# 28x28x1 means 28 rows, 28 columns and 1 channel
# The 1 channel will have values ranging from 0 to 1
# For RGB, we would have had a 3 channel system 

x_train = x_train.values.reshape(-1,28,28,1)
mnist_test = mnist_test.values.reshape(-1,28,28,1)
x_train.shape

# One Hot Encoding
Here, we'll convert our labels in y_train as a one-hot-encoded vector

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)

In [ ]:
print(y_train[3])

# Data Augmentation
It is performed in order to increase our dataset <br>
Commonly used methods are, random flipping, rotation, cropping, etc of images

In [ ]:
datagen = ImageDataGenerator(rotation_range=12, zoom_range = 0.15, width_shift_range=0.15, height_shift_range=0.13) 

datagen.fit(x_train)
datagen.fit(x_test)

# Visualize the Dataset
Here, we plot the first 20 entries of the dataset using matplotlib library

In [ ]:
num = 20
images = x_train[:num]

num_row = 5
num_col = 4

# plot images
fig, axes = plt.subplots(num_row, num_col, figsize=(1.5*num_col,2*num_row))
for i in range(num):
    ax = axes[i//num_col, i%num_col]
    ax.imshow(images[i].reshape(28,28), cmap='viridis')
    ax.set_title('Label: {}'.format(y_train[i].tolist().index(1.0, 0, 10)))
plt.tight_layout()
plt.show()

# Training and Validation set
Split the training data into training and validation sets

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.15)
print(x_train.shape)
print(x_val.shape)

# Convolutional Neural Network using Keras
Here, we build our learning model, which will be a convolutional neural network

In [ ]:
input_layer = tf.keras.layers.Input(shape = (28, 28, 1))

hidden_layer_1 = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation=tf.keras.activations.relu)(input_layer)
norm_1 = tf.keras.layers.BatchNormalization()(hidden_layer_1)
max_1 = tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2))(norm_1)
dropout_layer_1 = tf.keras.layers.Dropout(0.25)(max_1)

# hidden_layer_2 = tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation=tf.keras.activations.relu)(dropout_layer_1)
# norm_2 = tf.keras.layers.BatchNormalization()(hidden_layer_2)
# max_2 = tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2))(norm_2)

flatten_layer = tf.keras.layers.Flatten()(dropout_layer_1)
# dropout_layer_2 = tf.keras.layers.Dropout(0.25)(flatten_layer)

hidden_layer_3 = tf.keras.layers.Dense(256, activation=tf.keras.activations.relu)(flatten_layer)
hidden_layer_4 = tf.keras.layers.Dense(128, activation=tf.keras.activations.relu)(hidden_layer_3)

dropout_layer_3 = tf.keras.layers.Dropout(0.5)(hidden_layer_4)

output_layer = tf.keras.layers.Dense(10, activation=tf.keras.activations.sigmoid)(dropout_layer_3)

In [ ]:
model = tf.keras.Model(inputs = input_layer, outputs = output_layer)
model.summary()

# Plot of model graph

In [ ]:
tf.keras.utils.plot_model(model, "digit_classifier_model.png", show_shapes=True)

# Loading the model
Here, we load our CNN with optimizer, loss function and evaluation metric

In [ ]:
# Here, we are using Adam optimiser
# The loss function is categorical crossentropy function 

model.compile(optimizer = tf.keras.optimizers.Adam(), loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])

# Training our model
Here, we are training our **convolutional neural network** on the testing and validation data

In [ ]:
his = model.fit(x_train, y_train, batch_size = 1000, epochs = 40, validation_data=(x_val, y_val))

## Plotting model accuracy
We plot the model accuracies for both training and testing data

In [ ]:
plt.plot(his.history['accuracy'])
plt.plot(his.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

## Plotting model loss
We plot the model loss for both training and testing data

In [ ]:
plt.plot(his.history['loss'])
plt.plot(his.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

# Confusion Matrix
A confusion matrix can very well describe the performance of a classification model <br>
We'll plot one for our CNN model here

In [ ]:
# Lets plot the confusion matrix 

def plot_confusion_matrix(cm, classes,title='Confusion matrix',cmap=plt.cm.viridis):
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="black" if cm[i, j] > thresh else "white")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
y_pred = model.predict(x_val)
# Convert predictions classes to one hot vectors 
y_pred_classes = np.argmax(y_pred,axis = 1) 
# Convert validation observations to one hot vectors
y_true = np.argmax(y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(y_true, y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10)) 

# Displaying the error results and further analysis
So here, we are trying to display the digits which our model failed to recognize correctly <br>
Some of these are very trivial mistakes while the later ones are quite genuine and any human can easily mistake them for other digits just like our model did here

In [ ]:
# Display some error results 

# Errors are difference between predicted labels and true labels
errors = (y_pred_classes - y_true != 0)

y_pred_classes_errors = y_pred_classes[errors]
y_pred_errors = y_pred[errors]
y_true_errors = y_true[errors]
x_val_errors = x_val[errors]

def display_errors(errors_index,img_errors,pred_errors, obs_errors):
    
    n = 0
    nrows = 4
    ncols = 4
    fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True, figsize=(10,10))
    plt.subplots_adjust(wspace=0.5, hspace=0.5)
    for row in range(nrows):
        for col in range(ncols):
            error = errors_index[n]
            ax[row,col].imshow((img_errors[error]).reshape((28,28)))
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(pred_errors[error],obs_errors[error]))
            n += 1

# Probabilities of the wrong predicted numbers
y_pred_errors_prob = np.max(y_pred_errors,axis = 1)

# Predicted probabilities of the true values in the error set
true_prob_errors = np.diagonal(np.take(y_pred_errors, y_true_errors, axis=1))

# Difference between the probability of the predicted label and the true label
delta_pred_true_errors = y_pred_errors_prob - true_prob_errors

# Sorted list of the delta prob errors
sorted_dela_errors = np.argsort(delta_pred_true_errors)

# Top 16 errors 
most_important_errors = sorted_dela_errors[-16:]

# Show the top 16 errors
display_errors(most_important_errors, x_val_errors, y_pred_classes_errors, y_true_errors)

# Final Result
So here is the final result of our model :)

In [ ]:
# predict results
results = model.predict(mnist_test)
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("mnist_result.csv",index=False)
submission

I have looked up the web for a lot of help regarding my code, especially the confusion matrix and error display part, so it may appear somewhat similar for that matter <br>
Nevertheless, I was surprised as to how much one can learn from a very basic dataset such as MNIST and it was surely a nice way to recapitulate the concepts I've studied